In [1]:
from pipelines.alcohol import AlcoholPipeline

In [5]:
from data.dao import DataAccess, LabelGetter

In [8]:
from sklearn.cross_validation import train_test_split

In [11]:
from sklearn.linear_model import LogisticRegression

In [6]:
X = DataAccess.get_as_dataframe(explode=False)
ll = LabelGetter(X)

In [48]:
X, y = ll.get_alcohol()
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=23)

In [49]:

clf = AlcoholPipeline(global_features=["text"]).pipeline(LogisticRegression())

In [50]:
X.shape

(15650, 5)

In [81]:
X.sample(10).text

_id
55754014d6dfbb775fb51f76    peach Jell-O shots with vanilla vodka tastes p...
5584afa7d6dfbb636c07d80d    My favorite item on the menu...even though I c...
558efe04d6dfbb3d0e4c2990    Drinking  The Nothing (2015) by @SmogCityBeer ...
5577aa75d6dfbb5cd259e83c    I'm begging u please 😌 kause I can't swim that...
557f46abd6dfbb573b4d780b        OMGGGG @ginomortellaro http://t.co/hA4lVXhpSW
556bd56cd6dfbb3fac3ac001    @Chris_Kendall_ feel invited to the East Coast...
5584a6a8d6dfbb5faef789d1    😍😍😍 so stinking happy right meow. #jesterking ...
557b43b4d6dfbb2d94b9ff02                       Im soo happy to see Bale happy
55770d69d6dfbb179bf09f47    LA pride is this weekend and i've literALLY AL...
55727bc5d6dfbb5bf32d62ec    Some Jeremiah Red and strawberry margarita ple...
Name: text, dtype: object

In [51]:
%%time

clf.fit(X_train, y_train)

Wall time: 717 ms


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [52]:
X_train.head()

,created_at,labels,predict,text,user
_id,,,,,
558793f2d6dfbb38273cc35b,Mon Jun 22 04:49:55 +0000 2015,{'alcohol': 1},0.661905,then today got to see Jeremy and Sis and littl...,"{'favourites_count': 7, 'friends_count': 150, ..."
55827318d6dfbb3ad4e84fea,Thu Jun 18 07:28:24 +0000 2015,{'alcohol': 0},NaN,I mean didn't we just spend a whole week calli...,"{'favourites_count': 4441, 'friends_count': 38..."
55867a27d6dfbb1449ba6e8e,Sun Jun 21 08:47:36 +0000 2015,"{'first_person_level': 2, 'first_person': 1, '...",0.576796,It's an honor to receive a drunk call from me ...,"{'favourites_count': 3766, 'friends_count': 26..."
5578edefd6dfbb35dd86ca5c,Thu Jun 11 02:10:31 +0000 2015,{'alcohol': 0},0.197434,"Want to work in #WestCovina, CA? View our late...","{'favourites_count': 0, 'friends_count': 0, 'v..."
55907347d6dfbb3e7804f683,Sun Jun 28 22:20:50 +0000 2015,"{'first_person_level': 0, 'first_person': 1, '...",NaN,Lagunitas has great hops like always and the a...,"{'favourites_count': 3443, 'friends_count': 38..."


In [53]:
import pickle

In [54]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [55]:
y_predict = clf.predict(X_test)

In [56]:
print(accuracy_score(y_test, y_predict))
print(f1_score(y_test, y_predict))
print(confusion_matrix(y_test, y_predict))

0.835237173282
0.885510561012
[[1023  700]
 [ 151 3291]]


In [57]:
clf

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [59]:
pickle.dump(clf, open('clf_alc_test.p', 'wb'))

In [99]:
feature_weights = clf.steps[0][1].transformer_list[0][1].steps[1][1].idf_
sorted_weights = sorted(range(len(feature_weights)), key=lambda k: feature_weights[k])

In [102]:
features= clf.steps[0][1].transformer_list[0][1].steps[1][1].get_feature_names()

In [109]:
tfidf = clf.steps[0][1].transformer_list[0][1].steps[1][1]

In [104]:
import numpy as np

In [105]:
a_features = np.array(features)

In [108]:
a_features[sorted_weights][:10]

array(['co', 'the', 'and', 'to', 'http', 'drinking', 'drunk', 'https',
       'in', 'my'], 
      dtype='<U43')

In [110]:
tfidf.transform(r'"Dinner is ready!" "Okay, Coming!" *Stays on the computer* "DINNER IS READY!" "I SAID I AM COMING!!!!!!')

<103x23572 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>